# Multilabel Image Classification scenario with RAI Dashboard

The [fridge dataset](https://github.com/microsoft/computervision-recipes/tree/master/scenarios/classification) tags images with four types of items commonly found in the Microsoft New England R&D office refrigerator - carton, water bottle, can and milk bottle.  This example notebook demonstrates how to use a fine-tuned fastai multilabel computer vision model on the dataset to evaluate the model in AzureML.

First, we need to specify the version of the RAI components which are available in the workspace. This was specified when the components were uploaded.

In [ ]:
version_string = "0.0.5"

We also need to give the name of the compute cluster we want to use in AzureML. Later in this notebook, we will create it if it does not already exist:

In [ ]:
compute_name = "cpucluster"

Finally, we need to specify a version for the data and components we will create while running this notebook. This should be unique for the workspace, but the specific value doesn't matter:

In [ ]:
rai_example_version_string = "7"

## Accessing the Data

We supply the data as a pair of parquet files and accompanying `MLTable` file. We can download them, preprocess them, and take a brief look:

In [ ]:
import os
import pandas as pd
import json

try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve

First we download the fridge dataset:

In [ ]:
def download_fridge_dataset(data_path, annotations_file):
    os.makedirs(data_path, exist_ok=True)

    # download data
    base_url = "https://publictestdatasets.blob.core.windows.net/"
    fridge_folder = "computervision/multilabelFridgeObjects/"

    data_url = base_url + fridge_folder + annotations_file
    data_output_path = os.path.join(data_path, annotations_file)
    urlretrieve(data_url, filename=data_output_path)


test_annotations = "test_annotations.jsonl"

test_data_path = "multilabel_fridge_test_data"

download_fridge_dataset(test_data_path, test_annotations)

Now create the mltable:

In [ ]:
def create_ml_table_file(filename):
    return (
        "$schema: http://azureml/sdk-2-0/MLTable.json\n"
        "type: mltable\n"
        "paths:\n"
        " - file: ./{0}\n"
        "transformations:\n"
        "  - read_json_lines:\n"
        "        encoding: utf8\n"
        "        invalid_lines: error\n"
        "        include_path_column: false\n"
    ).format(filename)


def save_ml_table_file(output_path, ml_table_data):
    mltable_file_contents = create_ml_table_file(ml_table_data)
    with open(os.path.join(output_path, "MLTable"), "w") as f:
        f.write(mltable_file_contents)


save_ml_table_file(test_data_path, test_annotations)

Load some data for a quick view:

In [ ]:
import mltable

tbl = mltable.load(test_data_path)
test_df: pd.DataFrame = tbl.to_pandas_dataframe()

display(test_df)

The label columns contain the classes:

In [ ]:
target_column_name = ["can", "carton", "milk_bottle", "water_bottle"]
encoded_target_column_name = json.dumps(target_column_name)

First, we need to upload the datasets to our workspace. We start by creating an `MLClient` for interactions with AzureML:

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

try:
    ml_client = MLClient.from_config(
        credential=DefaultAzureCredential(exclude_shared_token_cache_credential=True),
        logging_enable=True,
    )
except Exception:
    # In case of exception due to config missing, try to get and create config, which may work if in compute instance
    from azureml.core import Workspace

    workspace = Workspace.from_config()
    workspace.write_config()
    ml_client = MLClient.from_config(
        credential=DefaultAzureCredential(exclude_shared_token_cache_credential=True),
        logging_enable=True,
    )

We can now upload the data to AzureML:

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_test_data = "Multilabel_Fridge_Test_MLTable"

test_data = Data(
    path=test_data_path,
    type=AssetTypes.MLTABLE,
    description="RAI Multilabel Fridge test data",
    name=input_test_data,
    version=rai_example_version_string,
)
ml_client.data.create_or_update(test_data)

# Creating the Model

To simplify the model creation process, we're going to use a pipeline.

We create a directory for the training script:

In [ ]:
import os

os.makedirs("fridge_component_src", exist_ok=True)

Next, we write out our script to retrieve the trained model:

In [ ]:
%%writefile fridge_component_src/training_script.py

import argparse
import logging
import json
import os
import time


import mlflow
import mlflow.pyfunc

from azureml.core import Run

from fastai.learner import load_learner

from raiutils.common.retries import retry_function

try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve

_logger = logging.getLogger(__file__)
logging.basicConfig(level=logging.INFO)

MULTILABEL_FRIDGE_MODEL_NAME = 'multilabel_fridge_model'


def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument(
        "--model_output_path", type=str, help="Path to write model info JSON"
    )
    parser.add_argument(
        "--model_base_name", type=str, help="Name of the registered model"
    )
    parser.add_argument(
        "--model_name_suffix", type=int, help="Set negative to use epoch_secs"
    )
    parser.add_argument(
        "--device", type=int, help=(
            "Device for CPU/GPU supports. Setting this to -1 will leverage "
            "CPU, >=0 will run the model on the associated CUDA device id.")
    )

    # parse args
    args = parser.parse_args()

    # return args
    return args


class FetchModel(object):
    def __init__(self):
        pass

    def fetch(self):
        url = ('https://publictestdatasets.blob.core.windows.net/models/' +
               MULTILABEL_FRIDGE_MODEL_NAME)
        urlretrieve(url, MULTILABEL_FRIDGE_MODEL_NAME)


def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    _logger.info("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)

    _logger.info("Getting device")
    device = args.device

    _logger.info("Loading parquet input")

    # Load the fridge fastai model
    fetcher = FetchModel()
    action_name = "Model download"
    err_msg = "Failed to download model"
    max_retries = 4
    retry_delay = 60
    retry_function(fetcher.fetch, action_name, err_msg,
                   max_retries=max_retries,
                   retry_delay=retry_delay)
    model = load_learner(MULTILABEL_FRIDGE_MODEL_NAME)

    if device >= 0:
        model = model.cuda()

    if args.model_name_suffix < 0:
        suffix = int(time.time())
    else:
        suffix = args.model_name_suffix
    registered_name = "{0}_{1}".format(args.model_base_name, suffix)
    _logger.info(f"Registering model as {registered_name}")

    # Saving model with mlflow
    _logger.info("Saving with mlflow")

    mlflow.fastai.log_model(
        model,
        artifact_path=registered_name,
        registered_model_name=registered_name
    )

    _logger.info("Writing JSON")
    dict = {"id": "{0}:1".format(registered_name)}
    output_path = os.path.join(args.model_output_path, "model_info.json")
    with open(output_path, "w") as of:
        json.dump(dict, fp=of)


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Now, we can build this into an AzureML component:

In [ ]:
from azure.ai.ml import load_component

yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: rai_multilabel_fridge_training_component
display_name: Multilabel Fridge training component for RAI example
version: {rai_example_version_string}
type: command
inputs:
  model_base_name:
    type: string
  model_name_suffix: # Set negative to use epoch_secs
    type: integer
    default: -1
  device: # set to >= 0 to use GPU
    type: integer
    default: 0
outputs:
  model_output_path:
    type: path
code: ./fridge_component_src/
environment: azureml://registries/azureml/environments/responsibleai-vision-ubuntu20.04-py38-cpu/versions/17
command: >-
  python training_script.py
  --model_base_name ${{{{inputs.model_base_name}}}}
  --model_name_suffix ${{{{inputs.model_name_suffix}}}}
  --device ${{{{inputs.device}}}}
  --model_output_path ${{{{outputs.model_output_path}}}}
"""

yaml_filename = "FridgeMultilabelVisionTrainingComp.yaml"

with open(yaml_filename, "w") as f:
    f.write(yaml_contents)

train_component_definition = load_component(source=yaml_filename)

ml_client.components.create_or_update(train_component_definition)

We need a compute target on which to run our jobs. The following checks whether the compute specified above is present; if not, then the compute target is created.

In [ ]:
from azure.ai.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if compute_name in all_compute_names:
    print(f"Found existing compute: {compute_name}")
else:
    my_compute = AmlCompute(
        name=compute_name,
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600,
    )
    ml_client.compute.begin_create_or_update(my_compute)
    print("Initiated compute creation")

## Running a training pipeline

Now that we have our training component, we can run it. We begin by generating a unique name for the model.

In [ ]:
import time

model_base_name = "multilabel_fridge_model"
model_name_suffix = "12454"
device = -1

Next, we define our training pipeline. This has two components. The first is the training component which we defined above. The second is a component to register the model in AzureML:

In [ ]:
from azure.ai.ml import dsl, Input

train_model_component = ml_client.components.get(
    name="rai_multilabel_fridge_training_component", version=rai_example_version_string
)


@dsl.pipeline(
    compute=compute_name,
    description="Register Model for RAI Multilabel Fridge example",
    experiment_name=f"RAI_Multilabel_Fridge_Example_Model_Training_{model_name_suffix}",
)
def my_training_pipeline(model_base_name, model_name_suffix, device):
    trained_model = train_component_definition(
        model_base_name=model_base_name,
        model_name_suffix=model_name_suffix,
        device=device,
    )
    trained_model.set_limits(timeout=120)

    return {}


model_registration_pipeline_job = my_training_pipeline(
    model_base_name, model_name_suffix, device
)

With the training pipeline defined, we can submit it for execution in AzureML. We define a helper function to wait for the job to complete:

In [ ]:
from azure.ai.ml.entities import PipelineJob


def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

## Creating the RAI Vision Insights

Now that we have our model, we can generate RAI Vision insights for it. We will need the `id` of the registered model, which will be as follows:

In [ ]:
expected_model_id = f"{model_base_name}_{model_name_suffix}:1"
azureml_model_id = f"azureml:{expected_model_id}"

Next, we load the RAI components, so that we can construct a pipeline:

In [ ]:
fridge_test_mltable = Input(
    type="mltable",
    path=f"{input_test_data}:{rai_example_version_string}",
    mode="download",
)

registry_name = "azureml"
credential = DefaultAzureCredential()

ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name,
)

rai_vision_insights_component = ml_client_registry.components.get(
    name="rai_vision_insights", version=version_string
)

We can now specify our pipeline. Complex objects (such as lists of column names) have to be converted to JSON strings before being passed to the components.

In [ ]:
import json
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes


@dsl.pipeline(
    compute=compute_name,
    description="Example RAI computation on Multilabel Fridge data",
    experiment_name=f"RAI_Multilable_Fridge_Example_RAIInsights_Computation_{model_name_suffix}",
)
def rai_fridge_image_classification_pipeline(target_column_name, test_data, classes):
    # Initiate the RAIInsights
    rai_image_job = rai_vision_insights_component(
        task_type="multilabel_image_classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        test_dataset=test_data,
        target_column_name=target_column_name,
        classes=classes,
        model_type="fastai",
    )
    rai_image_job.set_limits(timeout=120)

    rai_image_job.outputs.dashboard.mode = "upload"
    rai_image_job.outputs.ux_json.mode = "upload"

    return {
        "dashboard": rai_image_job.outputs.dashboard,
        "ux_json": rai_image_job.outputs.ux_json,
    }

Next, we define the pipeline object itself, and ensure that the outputs will be available for download:

In [ ]:
import uuid
from azure.ai.ml import Output

insights_pipeline_job = rai_fridge_image_classification_pipeline(
    target_column_name=encoded_target_column_name,
    test_data=fridge_test_mltable,
    classes="[]",
)

rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)
insights_pipeline_job.outputs.ux_json = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/ux_json/",
    mode="upload",
    type="uri_folder",
)

And submit the pipeline to AzureML for execution:

In [ ]:
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

The dashboard should appear in the AzureML portal in the registered model view. The following cell computes the expected URI:

In [ ]:
sub_id = ml_client._operation_scope.subscription_id
rg_name = ml_client._operation_scope.resource_group_name
ws_name = ml_client.workspace_name

expected_uri = f"https://ml.azure.com/model/{expected_model_id}/model_analysis?wsid=/subscriptions/{sub_id}/resourcegroups/{rg_name}/workspaces/{ws_name}"

print(f"Please visit {expected_uri} to see your analysis")

## Constructing the pipeline in YAML

It is also possible to specify the pipeline as a YAML file, and submit that using the command line. We will now create a YAML specification of the above pipeline and submit that:

In [ ]:
yaml_contents = f"""
$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
experiment_name: AML_RAI_Multilabel_Vision_Sample_{rai_example_version_string}
type: pipeline

compute: azureml:cpucluster

inputs:
  fridge_model_info: {expected_model_id}
  my_test_data:
    type: mltable
    path: azureml:{input_test_data}:{rai_example_version_string}
    mode: download

settings:
  default_datastore: azureml:workspaceblobstore
  default_compute: azureml:cpucluster
  continue_on_step_failure: false

jobs:
  analyse_model:
    type: command
    component: azureml://registries/azureml/components/rai_vision_insights/versions/{version_string}
    inputs:
      task_type: multilabel_image_classification
      model_input:
        type: mlflow_model
        path: {azureml_model_id}
      model_info: ${{{{parent.inputs.fridge_model_info}}}}
      test_dataset: ${{{{parent.inputs.my_test_data}}}}
      target_column_name: {encoded_target_column_name}
      maximum_rows_for_test_dataset: 5000
      classes: '[]'
"""

yaml_pipeline_filename = "rai_vision_example.yaml"

with open(yaml_pipeline_filename, "w") as f:
    f.write(yaml_contents)

The created file can then be submitted using the Azure CLI:

In [ ]:
cmd_line = [
    "az",
    "ml",
    "job",
    "create",
    "--resource-group",
    rg_name,
    "--workspace",
    ws_name,
    "--file",
    yaml_pipeline_filename,
]

import subprocess

try:
    cmd = subprocess.run(cmd_line, check=True, shell=True, capture_output=True)
except subprocess.CalledProcessError as cpe:
    print(f"Error invoking: {cpe.args}")
    print(cpe.stdout)
    print(cpe.stderr)
    raise
else:
    print("Azure CLI submission completed")